# Data Adjustment

Need a new data module to read and write files

In [ ]:
import os
import datetime
import numpy as np

from netCDF4 import Dataset

import chart

In [ ]:
class Sweep:
    def __init__(self, file=None):
        self.file = file
        if self.file:
            self.read()

    def read(self):
        with Dataset(self.file, mode='r') as nc:
            name = nc.getncattr('TypeName')
            self.elev = np.array(nc.variables['Elevation'][:], dtype=np.float32)
            self.azim = np.array(nc.variables['Azimuth'][:], dtype=np.float32)
            self.gatewidths = np.array(nc.variables['GateWidth'][:], dtype=np.float32)
            self.gatewidth = self.gatewidths[0]
            self.values = np.array(nc.variables[name][:], dtype=np.float32)
            self.values[self.values < -90] = np.nan
            self.longitude = nc.getncattr('Longitude')
            self.latitude = nc.getncattr('Latitude')
            self.sweepElev = nc.getncattr('Elevation')
            self.sweepTime = nc.getncattr('Time')
            self.symbol = self.file.split('.')[-2].split('-')[-1]

    def rangeInKilometers(self):
        r = 1.0e-3 * np.arange(self.values.shape[1]) * self.gatewidth
        return r
    
    def azimuthInRadians(self):
        a = np.deg2rad(self.azim)
        return a

    def titleString(self):
        t = datetime.datetime.utcfromtimestamp(self.sweepTime)
        timestr = t.strftime('%Y/%m/%d %H:%M:%S')
        title = f'{timestr} UTC  EL: {self.sweepElev:.2f}°'
        return title

In [ ]:
file = 'blob/PX-20170220-050706-E2.4-Z.nc'
sweep = Sweep(file)

In [ ]:
overlay = chart.atlas.Overlay((sweep.longitude, sweep.latitude))

In [ ]:
radii = np.concatenate(([1.0], np.arange(10.0, 61.0, 10.0)))
overlay.setRingRadii(radii)
overlay.load()

In [ ]:
r = sweep.rangeInKilometers()
a = sweep.azimuthInRadians()
title = sweep.titleString()
symbol = sweep.symbol

In [ ]:
ppi = chart.chart.Image(a, r, sweep.values, style=symbol, overlay=overlay, title=title, figsize=(800, 640), maxrange=50.0)

In [ ]:
ppi.image()